In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import malaya
import torch

torch.__version__

/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


'2.5.1+cu124'

In [11]:
model = malaya.true_case.huggingface(model = 'mesolitica/finetune-true-case-t5-small-standard-bahasa-cased')

tokenizer_config.json:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/803k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [12]:
_ = model.cuda()

In [13]:
string1 = 'masjid al takwa di miri udah nerima agih belanja seratus ribu ringgit ari opis menteri anegi teknologi hijau enggau ai kena ngitu pemanah masjid nya'
string2 = 'kuala lumpur menteri di jabatan perdana menteri datuk seri dr mujahid yusof rawa hari ini mengakhiri lawatan kerja lapan hari ke jordan turki dan bosnia herzegovina lawatan yang bertujuan mengeratkan lagi hubungan dua hala dengan ketiga tiga negara berkenaan'

In [14]:
model.generate([string1, string2], max_length = 256)

['Masjid Al-Takwa di Miri udah nerima agih belanja seratus ribu ringgit ari opis Menteri Anegi Teknologi Hijau Enggau Ai Kena ngitu pemanah masjid-nya.',
 'KUALA LUMPUR: Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua hala dengan ketiga-tiga negara berkenaan.']

In [16]:
from glob import glob

files = glob('iban/**/*_text', recursive = True)
files

['iban/data/train/train_text', 'iban/data/test/test_text']

In [19]:
lines = []
for f in files:
    with open(f) as fopen:
        t = fopen.read().split('\n')
        t = [t_ for t_ in t if len(t_)]
        lines.extend(t)
        
len(lines)

3132

In [37]:
from tqdm import tqdm

predicted = []
for l in tqdm(lines):
    splitted = l.split('  ', 1)
    if len(splitted) == 2 and len(splitted[1]) < 1:
        continue
    
    r = model.generate([splitted[1]], max_length = 256)
    predicted.append({
        'id': splitted[0],
        'original': splitted[1],
        'true_case': r[0]
    })

100%|███████████████████████████████████████| 3132/3132 [08:46<00:00,  5.95it/s]


In [39]:
import json

with open('true-case.json', 'w') as fopen:
    json.dump(predicted, fopen)

In [40]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="true-case.json",
    path_in_repo="true-case.json",
    repo_id="malaysia-ai/iban-whisper-format",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/iban-whisper-format/commit/865cbbdf1fcc6dac1cc381dd7c5377c470da6d31', commit_message='Upload true-case.json with huggingface_hub', commit_description='', oid='865cbbdf1fcc6dac1cc381dd7c5377c470da6d31', pr_url=None, pr_revision=None, pr_num=None)